# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""../../../../data/SampleHDS.jso…",1.8193e19,"""8734997962293760244""","""5053529787726237372""","""3565094847481399216""","""14926335563951212260""",false
"""../../../../data/SampleHDS.jso…",1.2841e19,"""9165622199217652805""","""11676944598815932332""","""3565094847481399216""","""10692743611995956266""",true
"""../../../../data/SampleHDS.jso…",9.0346e18,"""5470981165119945971""","""18346501983323618001""","""488021697381017042""","""14926335563951212260""",false
"""../../../../data/SampleHDS.jso…",3.4840e18,"""9165622199217652805""","""8833922649591212132""","""13078763322118606591""","""14926335563951212260""",false
"""../../../../data/SampleHDS.jso…",1.2816e19,"""8734997962293760244""","""17250062085705149590""","""13078763322118606591""","""9719888628032625823""",true
"""../../../../data/SampleHDS.jso…",1.6556e19,"""5470981165119945971""","""7350018321995118263""","""16462107798657238381""",null,false
"""../../../../data/SampleHDS.jso…",4.2111e18,"""5470981165119945971""","""18163767353946447002""","""13078763322118606591""","""9719888628032625823""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../data/SampleHDS.jso…",1.8367e19,"""10014215516237642637""","""3545409135666671446""","""FirstMortgage30yr""","""8307260166483955823""","""Rejected"""
"""../../../../data/SampleHDS.jso…",7.8588e18,"""3101982415141958751""","""16704935892369544798""","""FirstMortgage30yr""","""12255394379751775767""","""Accepted"""
"""../../../../data/SampleHDS.jso…",6.4746e17,"""4998695217353721849""","""2554918563875994690""","""MoneyMarketSavingsAccount""","""8307260166483955823""","""Rejected"""
"""../../../../data/SampleHDS.jso…",1.8739e18,"""3101982415141958751""","""18117355864429940457""","""BasicChecking""","""8307260166483955823""","""Rejected"""
"""../../../../data/SampleHDS.jso…",1.0661e18,"""10014215516237642637""","""15416208327096336538""","""BasicChecking""","""14383912147680247276""","""Accepted"""
"""../../../../data/SampleHDS.jso…",1.5476e19,"""4998695217353721849""","""5736487611059122044""","""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…",6.7072e18,"""4998695217353721849""","""16176191152568588281""","""BasicChecking""","""14383912147680247276""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../data/SampleHDS.jso…",1.0712e19,"""8031211062353165529""","""8811824235266848814""","""FirstMortgage30yr""","""12140100940265661492""","""Rejected"""
"""../../../../data/SampleHDS.jso…",1.5625e19,"""210841704404280211""","""2214160391327182345""","""FirstMortgage30yr""","""11332604317151177476""","""Accepted"""
"""../../../../data/SampleHDS.jso…",1.6160e19,"""14698368787774945681""","""7785060098416983163""","""MoneyMarketSavingsAccount""","""12140100940265661492""","""Rejected"""
"""../../../../data/SampleHDS.jso…",1.5378e19,"""210841704404280211""","""11766921491014743946""","""BasicChecking""","""12140100940265661492""","""Rejected"""
"""../../../../data/SampleHDS.jso…",3.0819e18,"""8031211062353165529""","""9686015612093897294""","""BasicChecking""","""8755663088505372483""","""Accepted"""
"""../../../../data/SampleHDS.jso…",6.1597e18,"""14698368787774945681""","""6620634759934612375""","""UPlusFinPersonal""",null,"""Rejected"""
"""../../../../data/SampleHDS.jso…",1.0319e19,"""14698368787774945681""","""15401600220661770765""","""BasicChecking""","""8755663088505372483""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_CLV=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
3.9301e18,"""9193919666310739798""","""2415626730374262795""","""FirstMortgage30yr""","""16984521334003788096""","""Rejected"""
1.3039e19,"""16937815534728044849""","""4995419402162828313""","""FirstMortgage30yr""","""9590627531974522891""","""Accepted"""
1.3724e19,"""12806974844925541""","""6011465764692857178""","""MoneyMarketSavingsAccount""","""16984521334003788096""","""Rejected"""
1.2369e19,"""16937815534728044849""","""5246301241847455799""","""BasicChecking""","""16984521334003788096""","""Rejected"""
1.8141e18,"""9193919666310739798""","""16325690720798853039""","""BasicChecking""","""11606300632664520659""","""Accepted"""
1.8004e19,"""12806974844925541""","""15430680152427342239""","""UPlusFinPersonal""",null,"""Rejected"""
6.3356e18,"""12806974844925541""","""3633733981053265685""","""BasicChecking""","""11606300632664520659""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'